In [20]:
import pandas as pd
import numpy as np
import math

In [21]:
master_df = pd.read_excel('data/OCFF-2021_Master-List_01-04-21.xlsx', sheet_name=1, engine='openpyxl')

In [22]:
master_df['Duration'] = pd.to_timedelta(master_df['Duration'])

In [23]:
master_df['Duration (Min)'] = round((master_df['Duration'] / np.timedelta64(1, 'm')), ndigits=2)

In [29]:
blocks = {'Free': 0, 'Shorts': 0, 'Features': 0}

for minutes in master_df['Duration (Min)']:
    if minutes < 3:
        blocks['Free'] += 1
    elif minutes >= 3 and minutes < 30:
        blocks['Shorts'] += 1
    else:
        blocks['Features'] += 1

print(f'Under 3 mins: {blocks["Free"]} * 0 = $0')
print(f'Over 3 mins and under 30 mins: {blocks["Shorts"]} * $5 = ${(blocks["Shorts"] - 20) * 5}')
print(f'Over 30 mins: {blocks["Features"]} (+ 6) * $25: ${((blocks["Features"] - 10) + 6) * 25}')

Under 3 mins: 5 * 0 = $0
Over 3 mins and under 30 mins: 59 * $5 = $195
Over 30 mins: 8 (+ 6) * $25: $100


In [25]:
duration_df = master_df.loc[:, ('Submission Categories', 'Duration (Min)')]
duration_per_category = duration_df.groupby('Submission Categories').sum()
duration_per_category['Count'] = duration_df.groupby('Submission Categories').count()

In [26]:
duration_per_category[f'30 Min Blocks'] = (round((duration_per_category.loc[:, 'Duration (Min)'] / 30), ndigits=2)).apply(lambda x: math.ceil(x))
duration_per_category['60 Min Blocks'] = (round((duration_per_category.loc[:, 'Duration (Min)'] / 60), ndigits=2)).apply(lambda x: math.ceil(x))
duration_per_category['90 Min Blocks'] = (round((duration_per_category.loc[:, 'Duration (Min)'] / 90), ndigits=2)).apply(lambda x: math.ceil(x))

In [27]:
duration_per_category.loc['TOTALS'] = duration_per_category.sum()

In [28]:
duration_per_category

,Duration (Min),Count,30 Min Blocks,60 Min Blocks,90 Min Blocks
Submission Categories,,,,,
Animation,50.53,11.0,2.0,1.0,1.0
Dramatic Shorts,132.36,11.0,5.0,3.0,2.0
Environmental & Aquatic Films,78.88,6.0,3.0,2.0,1.0
Feature Films,562.07,9.0,19.0,10.0,7.0
Funny Shorts,90.19,7.0,4.0,2.0,1.0
Horror/Sci-fi,121.50,11.0,5.0,3.0,2.0
Short Documentary,116.79,7.0,4.0,2.0,2.0
Short films that make you feel good.,55.07,5.0,2.0,1.0,1.0
Youth Short Films,31.10,5.0,2.0,1.0,1.0


In [30]:
duration_per_category.to_excel('data/Cost-Analysis.xlsx')